In [1]:
# Import Splinter, BeautifulSoup an Pandas.

from splinter import Browser

from bs4 import BeautifulSoup as soup

from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd

import datetime as dt

In [2]:
# Set up Splinter

executable_path = {'executable_path': ChromeDriverManager().install()}

browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Visit the mars nasa news site

url = 'https://redplanetscience.com'

browser.visit(url)

# Optional delay for loading the page

browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
# Convert the browser html to a soup object.

html = browser.html

news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('div.list_text')

In [5]:
slide_elem.find('div', class_='content_title')

<div class="content_title">HiRISE Views NASA's InSight and Curiosity on Mars</div>

In [6]:
# Use the parent element to find the first `a` tag and save it as `news_title`

news_title = slide_elem.find('div', class_='content_title').get_text()

news_title

"HiRISE Views NASA's InSight and Curiosity on Mars"

In [7]:
# Use the parent element to find the paragraph text

news_p = slide_elem.find('div', class_='article_teaser_body').get_text()

news_p

'New images taken from space offer the clearest orbital glimpse yet of InSight as well as a view of Curiosity rolling along.'

### Featured Images

In [8]:
# Visit URL

url = 'https://spaceimages-mars.com'

browser.visit(url)

In [9]:
# Find and click the full image button

full_image_elem = browser.find_by_tag('button')[1]

full_image_elem.click()

In [10]:
# Parse the resulting html with soup

html = browser.html

img_soup = soup(html, 'html.parser')

In [11]:
# Find the relative image url

img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')

img_url_rel

'image/featured/mars3.jpg'

In [12]:
# Use the base URL to create an absolute URL

img_url = f'https://spaceimages-mars.com/{img_url_rel}'

img_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

In [13]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]

df.columns=['description', 'Mars', 'Earth']

df.set_index('description', inplace=True)

df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [15]:
#browser.quit()

In [16]:
# Import Splinter, BeautifulSoup, and Pandas

from splinter import Browser

from bs4 import BeautifulSoup as soup

import pandas as pd

import datetime as dt

from webdriver_manager.chrome import ChromeDriverManager


def scrape_all():

    # Initiate headless driver for deployment
    
    executable_path = {'executable_path': ChromeDriverManager().install()}
    
    browser = Browser('chrome', **executable_path, headless=True)

    news_title, news_paragraph = mars_news(browser)

    
    # Run all scraping functions and store results in a dictionary
    
    data = {
        
        "news_title": news_title,
        
        "news_paragraph": news_paragraph,
        
        "featured_image": featured_image(browser),
        
        "facts": mars_facts(),
        
        "last_modified": dt.datetime.now()
        
    }

    
    # Stop webdriver and return data
    
    browser.quit()
    
    return data


def mars_news(browser):

    
    # Scrape Mars News
    
    # Visit the mars nasa news site
    
    url = 'https://data-class-mars.s3.amazonaws.com/Mars/index.html'
    
    browser.visit(url)

    
    # Optional delay for loading the page
    
    browser.is_element_present_by_css('div.list_text', wait_time=1)

    
    # Convert the browser html to a soup object and then quit the browser
    
    html = browser.html
    
    news_soup = soup(html, 'html.parser')

    
    # Add try/except for error handling
    
    try:
        
        slide_elem = news_soup.select_one('div.list_text')
        
        
        # Use the parent element to find the first 'a' tag and save it as 'news_title'
        
        news_title = slide_elem.find('div', class_='content_title').get_text()
        
        
        # Use the parent element to find the paragraph text
        
        news_p = slide_elem.find('div', class_='article_teaser_body').get_text()

        
    except AttributeError:
        
        return None, None

    return news_title, news_p


def featured_image(browser):
    
    # Visit URL
    
    url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
    
    browser.visit(url)

    
    # Find and click the full image button
    
    full_image_elem = browser.find_by_tag('button')[1]
    
    full_image_elem.click()

    
    # Parse the resulting html with soup
    
    html = browser.html
    
    img_soup = soup(html, 'html.parser')

    
    # Add try/except for error handling
    
    try:
        
        # Find the relative image url
        
        img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')

        
    except AttributeError:
        
        return None

    
    # Use the base url to create an absolute url
    
    img_url = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{img_url_rel}'

    
    return img_url


def mars_facts():
    
    # Add try/except for error handling
    
    try:
        
        # Use 'read_html' to scrape the facts table into a dataframe
        
        df = pd.read_html('https://data-class-mars-facts.s3.amazonaws.com/Mars_Facts/index.html')[0]

        
    except BaseException:
        
        return None

    
    # Assign columns and set index of dataframe
    
    df.columns=['Description', 'Mars', 'Earth']
    
    df.set_index('Description', inplace=True)

    
    # Convert dataframe into HTML format, add bootstrap
    
    return df.to_html(classes="table table-striped")


if __name__ == "__main__":

    # If running as script, print scraped data
    
    print(scrape_all())

{'news_title': '7 Things to Know About the Mars 2020 Perseverance Rover Mission', 'news_paragraph': "NASA's next rover to the Red Planet is slated to launch no earlier than July 30. These highlights will get you up to speed on the ambitious mission.", 'featured_image': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg', 'facts': '<table border="1" class="dataframe table table-striped">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2